In [21]:
import json
import lightgbm as lgb
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


In [22]:
df_train = pd.read_csv('first_round_training_data.csv')
df_test = pd.read_csv('first_round_testing_data.csv')

In [23]:
dit = {'Excellent':0,'Good':1,'Pass':2,'Fail':3}
df_train['label'] = df_train['Quality_label'].map(dit)
# 将特征转换成数字

In [24]:
df_train=df_train.drop(labels=['Attribute1','Attribute2','Attribute3','Attribute4','Attribute5','Attribute6','Attribute7',
                       'Attribute8','Attribute9','Attribute10','Quality_label', ],axis=1)

In [25]:
data=df_train.drop(labels='label',axis=1)
target = df_train['label']
X_train,X_test,y_train,y_test =train_test_split(data,target,test_size=0.2)


In [31]:

# 创建成lgb特征的数据集格式
lgb_train = lgb.Dataset(X_train, y_train) # 将数据保存到LightGBM二进制文件将使加载更快
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)  # 创建验证数据
 
# 将参数写成字典下形式
params = {
    'learning_rate':0.05,
    'lambda_l1':0.1,
    'lambda_l2':0.2,
    'max_depth':5,
    'objective':'multiclass',
    'num_class':4,  #lightgbm.basic.LightGBMError: b‘Number of classes should be specified and greater than 1 for multiclass training‘
}
 
print('Start training...')
# 训练 cv and train
gbm = lgb.train(params,lgb_train,num_boost_round=200,valid_sets=lgb_eval,early_stopping_rounds=5) # 训练数据需要参数列表和数据集
 


Start training...
[1]	valid_0's multi_logloss: 1.30912
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.29615
[3]	valid_0's multi_logloss: 1.28405
[4]	valid_0's multi_logloss: 1.27348
[5]	valid_0's multi_logloss: 1.26326
[6]	valid_0's multi_logloss: 1.25422
[7]	valid_0's multi_logloss: 1.24524
[8]	valid_0's multi_logloss: 1.23782
[9]	valid_0's multi_logloss: 1.23001
[10]	valid_0's multi_logloss: 1.22366
[11]	valid_0's multi_logloss: 1.21712
[12]	valid_0's multi_logloss: 1.21124
[13]	valid_0's multi_logloss: 1.20624
[14]	valid_0's multi_logloss: 1.20108
[15]	valid_0's multi_logloss: 1.19663
[16]	valid_0's multi_logloss: 1.19192
[17]	valid_0's multi_logloss: 1.18767
[18]	valid_0's multi_logloss: 1.18336
[19]	valid_0's multi_logloss: 1.17977
[20]	valid_0's multi_logloss: 1.1764
[21]	valid_0's multi_logloss: 1.17314
[22]	valid_0's multi_logloss: 1.16994
[23]	valid_0's multi_logloss: 1.16729
[24]	valid_0's multi_logloss: 1.16452
[25]	valid_0's mult

In [27]:

# y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration) #如果在训练期间启用了早期停止，可以通过best_iteration方式从最佳迭代中获得预测
# # 评估模型
# print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5) # 计算真实值和预测值之间的均方根误差


In [32]:
example = pd.read_csv('submit_example.csv')

In [38]:
prediction=gbm.predict(df_test.drop(labels='Group',axis=1))
prediction

array([[0.15876517, 0.4125143 , 0.31069304, 0.11802749],
       [0.06380517, 0.19738477, 0.51739864, 0.22141142],
       [0.0894644 , 0.30004762, 0.43975718, 0.17073081],
       ...,
       [0.05469906, 0.12360468, 0.18400854, 0.63768772],
       [0.07618368, 0.21920376, 0.52555766, 0.1790549 ],
       [0.13706077, 0.36636144, 0.41454186, 0.08203593]])

In [37]:
prediction.shape

(6000, 4)

In [39]:
# df_test

In [35]:
df_test['label'] = prediction
result = pd.DataFrame({'Group':example.Group})

ValueError: Wrong number of items passed 4, placement implies 1

In [ ]:
prediction

In [ ]:
for group_num in range(120):
#     print(group_num)
    all_counts = df_test.loc[df_test['Group'] ==group_num]['label'].value_counts()
#     print(all_counts)
    try:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = all_counts[0]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Excellent ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Good ratio'] = all_counts[1]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Good ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = all_counts[2]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Pass ratio'] = 0
    try:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = all_counts[3]/50
    except Exception as e:
        result.loc[result['Group'] ==group_num,'Fail ratio'] = 0

In [ ]:
result.to_csv(r'result.csv',index=False)